# Initial Setup: Install Spark

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.2/spark-2.3.2-bin-hadoop2.7.tgz
!tar xf spark-2.3.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.2-bin-hadoop2.7"

## Create a new Spark Session

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Test if the Spark context is running properly:

In [4]:
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)

/content/spark-2.3.2-bin-hadoop2.7/python/pyspark/sql/session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows





---

Click the arrow on the left and navigate to files to upload your demographics.csv and transactions.csv files.

Load the transactions and demographics table as dataframes in your spark cluster:

In [0]:
demographics = spark.read.load('demographics.csv', format="csv", header="true", sep=',', inferSchema=True)
transactions = spark.read.load('transactions.csv', format="csv", header="true", sep=',', inferSchema=True)

Create tables from dataframes:

In [0]:
demographics.write.saveAsTable("demographics")
transactions.write.saveAsTable("transactions")

In [7]:
spark.catalog.listTables()

[Table(name='demographics', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='transactions', database='default', description=None, tableType='MANAGED', isTemporary=False)]

### Get a table from the cluster

In [8]:
spark.sql("SELECT * FROM demographics").show()

+--------+--------+------+----------+----------+-------+---------+---------+------------+-------------+
|     ZIP|Customer|Gender| Birthdate|  JoinDate|zip_zip| zip_city|zip_state|zip_latitude|zip_longitude|
+--------+--------+------+----------+----------+-------+---------+---------+------------+-------------+
|US-00604|  111025|     f|28.04.1964|02.08.1980|    604|Aguadilla|       PR|   18.498987|    -67.13699|
|US-00604|  176830|     f|18.04.1994|24.03.2008|    604|Aguadilla|       PR|   18.498987|    -67.13699|
|US-00612|  100320|     f|04.04.1993|17.12.2004|    612|  Arecibo|       PR|   18.449732|    -66.69879|
|US-00612|  162212|     m|03.05.1997|11.12.2010|    612|  Arecibo|       PR|   18.449732|    -66.69879|
|US-00612|  174360|     f|23.08.1975|30.09.1993|    612|  Arecibo|       PR|   18.449732|    -66.69879|
|US-00612|  184182|     f|06.03.1955|13.01.1971|    612|  Arecibo|       PR|   18.449732|    -66.69879|
|US-00613|  117484|     f|23.07.1993|03.12.2007|    613|  Arecib

### Create a new table in the cluster

In [0]:
demographics.write.saveAsTable("demographics_copy")

### Drop a table

In [10]:
spark.sql("DROP table demographics_copy")

DataFrame[]

### Close connection

In [0]:
spark.stop()

Create a Pandas DataFrame from a Spark Dataframe:

In [0]:
demographicspandas = demographics.toPandas()

# Handling Spark DataFrames
## Select
### Select rows by condition

In [11]:
spark.sql("SELECT * FROM transactions").show()

+--------+----------+--------+-----------+-----+---------+--------+
|Customer| TransDate|Quantity|PurchAmount| Cost|  TransID|TransKey|
+--------+----------+--------+-----------+-----+---------+--------+
|  149332|15.11.2005|       1|     199.95|107.0|127998739|  100000|
|  172951|29.08.2008|       1|     199.95|108.0|128888288|  100001|
|  120621|19.10.2007|       1|      99.95| 49.0|125375247|  100002|
|  149236|14.11.2005|       1|      39.95|18.95|127996226|  100003|
|  149236|12.06.2007|       1|      79.95| 35.0|128670302|  100004|
|  140729|19.11.2009|       1|     129.95| 59.0|127637750|  100005|
|  140729|19.11.2009|       1|      79.95|39.18|127637750|  100006|
|  140729|19.11.2009|       1|      24.95| 11.5|127637750|  100007|
|  140729|19.11.2009|       1|       7.95| 2.96|127637750|  100008|
|  140729|28.04.2012|       1|      89.95| 35.0|129377737|  100009|
|  140729|28.04.2012|       1|      69.95| 26.4|129377737|  100010|
|  180970|19.06.2009|       1|     119.95|58.46|

### Combine operations to select rows and columns

In [0]:
transactions.filter("PurchAmount > 100").select("TransDate","Cost").show()

+----------+-----+
| TransDate| Cost|
+----------+-----+
|15.11.2005|107.0|
|29.08.2008|108.0|
|19.11.2009| 59.0|
|19.06.2009|58.46|
|11.02.2009|63.08|
|29.08.2008|162.5|
|29.08.2008|187.5|
|11.05.2008| 58.6|
|18.12.2006| 61.1|
|21.05.2008|92.89|
|30.04.2010|160.0|
|30.03.2006| 44.5|
|12.04.2006|80.96|
|21.08.2009| 68.0|
|12.08.2011|50.71|
|19.12.2011|51.31|
|06.08.2011| 65.0|
|06.08.2011|108.0|
|06.08.2011| 50.5|
|13.11.2009| 93.0|
+----------+-----+
only showing top 20 rows



## Aggregate

In [0]:
spark.sql("SELECT Customer, \
                  SUM(PurchAmount) AS AggPurch, \
                  SUM(Quantity) AS AggQuant \
           FROM transactions \
           GROUP BY Customer").show()

+--------+------------------+--------+
|Customer|          AggPurch|AggQuant|
+--------+------------------+--------+
|  146224|             64.95|       1|
|  196301|             79.95|       1|
|  198591|             119.9|       2|
|  193640|189.84999999999997|       3|
|  171723|254.84999999999997|       3|
|  139469|             169.9|       2|
|  154033|             24.95|       1|
|  144991|              54.9|       2|
|  173059|             99.85|       3|
|  171946|            299.95|       1|
|  192545|              54.9|       2|
|  198430|             169.9|       2|
|  147524|             569.7|       6|
|  194478|            214.85|       3|
|  151871|289.84999999999997|       3|
|  147961|             59.95|       1|
|  188488|              59.9|       2|
|  171006|             49.95|       1|
|  178564|            139.95|       1|
|  188122|            169.95|       1|
+--------+------------------+--------+
only showing top 20 rows



## Joins

In [0]:
spark.sql("SELECT * \
           FROM transactions \
           INNER JOIN demographics \
 	         ON transactions.Customer=demographics.Customer \
	         AND \
	         transactions.TransDate = demographics.JoinDate").show()

+--------+----------+--------+-----------+-----+---------+--------+--------+--------+------+----------+----------+-------+------------------+---------+------------+-------------+
|Customer| TransDate|Quantity|PurchAmount| Cost|  TransID|TransKey|     ZIP|Customer|Gender| Birthdate|  JoinDate|zip_zip|          zip_city|zip_state|zip_latitude|zip_longitude|
+--------+----------+--------+-----------+-----+---------+--------+--------+--------+------+----------+----------+-------+------------------+---------+------------+-------------+
|  170823|04.04.2008|       2|      109.9| 45.2|128786894|  100214|US-99515|  170823|     m|21.03.1998|04.04.2008|  99515|         Anchorage|       AK|   61.122943|   -149.88852|
|  103853|07.06.2006|       1|     159.95|81.11|124416583|  101154|US-94114|  103853|     f|10.08.1990|07.06.2006|  94114|     San Francisco|       CA|   37.758434|   -122.43512|
|  102521|14.09.2005|       1|     159.95|120.0|124386341|  101521|US-94803|  102521|     m|19.07.1996|14